In [2]:
!cat /proc/meminfo

MemTotal:       65951648 kB
MemFree:        62812820 kB
MemAvailable:   64356788 kB
Buffers:          655484 kB
Cached:          1299436 kB
SwapCached:            0 kB
Active:          1755068 kB
Inactive:         867048 kB
Active(anon):     667200 kB
Inactive(anon):       60 kB
Active(file):    1087868 kB
Inactive(file):   866988 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                84 kB
Writeback:             4 kB
AnonPages:        667496 kB
Mapped:           146236 kB
Shmem:                72 kB
Slab:             356912 kB
SReclaimable:     314008 kB
SUnreclaim:        42904 kB
KernelStack:        8256 kB
PageTables:         7376 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    32975824 kB
Committed_AS:    2897148 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [1]:
import pandas as pd
import numpy as np
import io
import boto3
from sklearn import preprocessing as prp

# custom functions and objects
from utils import *
# from utils.sample_strategies import stg2_sample, stg2_period_samples, stg2_sample_exclude

# ruta de trabajo en s3
bucket = 'boi-banregio'
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV'

# s3 resource
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

impute_strategy = 'median'

## Preprocess strategy 1

In [4]:
data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/STRATEGY1.csv"
data_file_key = "{}/{}".format(prefix, data_file_name)
data_obj = s3_bucket_resource.Object(data_file_key).get()
data = pd.read_csv(io.BytesIO(data_obj['Body'].read()))

In [5]:
# columns with to much missing

allowed_missing = 0.4
# count = 0
missing_cols = []
for col in data.columns:
    if data.loc[:,col].isnull().values.sum() > len(data) * allowed_missing:
        # print(col, count)
        missing_cols.append(col)
    # count += 1
print(len(missing_cols))

1991


In [6]:
# data without missings
data_wom = data.drop(missing_cols, axis=1) 

In [7]:
imputer = prp.Imputer(missing_values='NaN', strategy=impute_strategy, axis=0)
scaler = prp.RobustScaler()
prp_data = data_wom.drop(['LABEL', 'BMI', 'REVOLVENTE'], axis=1)

In [8]:
imputer = imputer.fit(prp_data)
data_wom.iloc[:,3:] = imputer.transform(prp_data)

In [9]:
scaler = scaler.fit(data_wom.iloc[:,3:])
data_wom.iloc[:,3:] = scaler.transform(data_wom.iloc[:,3:])

In [10]:
import pickle

pickle.dump(missing_cols, open("pickles/stg1_missing_cols.data", "wb"))
pickle.dump(imputer, open("pickles/stg1_imputer.data", "wb"))
pickle.dump(scaler, open("pickles/stg1_scaler.data", "wb"))

In [11]:
f_str = io.StringIO()
data_wom.to_csv(f_str, index=False)
data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/STRATEGY1_PRP.csv"
data_obj_key =  "{}/{}".format(prefix, data_file_name)
s3_bucket_resource.Object(data_obj_key).put(Body=f_str.getvalue())

{'ResponseMetadata': {'RequestId': 'D32BD3F36F2378DD',
  'HostId': 'Psw6BPqZ/nz8XJl5BKC23jXni5UcSWu33pHZj8xtU53Qk61hkj6S6VOFIzW0wc2/ebJZYqSgvN0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Psw6BPqZ/nz8XJl5BKC23jXni5UcSWu33pHZj8xtU53Qk61hkj6S6VOFIzW0wc2/ebJZYqSgvN0=',
   'x-amz-request-id': 'D32BD3F36F2378DD',
   'date': 'Tue, 06 Nov 2018 23:57:10 GMT',
   'etag': '"c3b86cc713c7ff0643a0771e37973fd0"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"c3b86cc713c7ff0643a0771e37973fd0"'}

## Preprocess strategy 1 all variables

In [2]:
data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/STRATEGY1.csv"
data_file_key = "{}/{}".format(prefix, data_file_name)
data_obj = s3_bucket_resource.Object(data_file_key).get()
data = pd.read_csv(io.BytesIO(data_obj['Body'].read()))

In [3]:
imputer = prp.Imputer(missing_values='NaN', strategy=impute_strategy, axis=0)
scaler = prp.RobustScaler()
prp_data = data.drop(['LABEL', 'BMI', 'REVOLVENTE'], axis=1)

In [4]:
imputer = imputer.fit(prp_data)
data.iloc[:,3:] = imputer.transform(prp_data)

In [6]:
scaler = scaler.fit(data.iloc[:,3:])
data.iloc[:,3:] = scaler.transform(data.iloc[:,3:])

In [7]:
import pickle

# pickle.dump(missing_cols, open("pickles/stg1_all_missing_cols.data", "wb"))
pickle.dump(imputer, open("pickles/stg1_all_imputer.data", "wb"))
pickle.dump(scaler, open("pickles/stg1_all_scaler.data", "wb"))

In [8]:
f_str = io.StringIO()
data.to_csv(f_str, index=False)
data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/STRATEGY1_ALL_PRP.csv"
data_obj_key =  "{}/{}".format(prefix, data_file_name)
s3_bucket_resource.Object(data_obj_key).put(Body=f_str.getvalue())

{'ResponseMetadata': {'RequestId': '2D5CD609328DEBEE',
  'HostId': 'eyhtv7SNhhI0ccoFSY3XpIHKlo34V6+r1aObG+l91d/xJt/C6RWDtw1XR4DQ7i2QB48Hldtoe0M=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'eyhtv7SNhhI0ccoFSY3XpIHKlo34V6+r1aObG+l91d/xJt/C6RWDtw1XR4DQ7i2QB48Hldtoe0M=',
   'x-amz-request-id': '2D5CD609328DEBEE',
   'date': 'Wed, 07 Nov 2018 19:02:20 GMT',
   'etag': '"a448f06440f938539aa0928c262e6578"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"a448f06440f938539aa0928c262e6578"'}

## Preprocess strategy 2

In [12]:
data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/STRATEGY2.csv"
data_file_key = "{}/{}".format(prefix, data_file_name)
data_obj = s3_bucket_resource.Object(data_file_key).get()
data = pd.read_csv(io.BytesIO(data_obj['Body'].read()))

In [13]:
# columns with to much missing

allowed_missing = 0.4
# count = 0
missing_cols = []
for col in data.columns:
    if data.loc[:,col].isnull().values.sum() > len(data) * allowed_missing:
        # print(col, count)
        missing_cols.append(col)
    # count += 1
print(len(missing_cols))

1991


In [14]:
# data without missings
data_wom = data.drop(missing_cols, axis=1) 

In [15]:
imputer = prp.Imputer(missing_values='NaN', strategy=impute_strategy, axis=0)
scaler = prp.RobustScaler()
prp_data = data_wom.drop(['LABEL', 'BMI', 'REVOLVENTE'], axis=1)

In [16]:
imputer = imputer.fit(prp_data)
data_wom.iloc[:,3:] = imputer.transform(prp_data)

In [17]:
scaler = scaler.fit(data_wom.iloc[:,3:])
data_wom.iloc[:,3:] = scaler.transform(data_wom.iloc[:,3:])

In [18]:
import pickle

pickle.dump(missing_cols, open("pickles/stg2_missing_cols.data", "wb"))
pickle.dump(imputer, open("pickles/stg2_imputer.data", "wb"))
pickle.dump(scaler, open("pickles/stg2_scaler.data", "wb"))

In [19]:
f_str = io.StringIO()
data_wom.to_csv(f_str, index=False)
data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/STRATEGY2_PRP.csv"
data_obj_key =  "{}/{}".format(prefix, data_file_name)
s3_bucket_resource.Object(data_obj_key).put(Body=f_str.getvalue())

{'ResponseMetadata': {'RequestId': '3F2BB1912F5427B8',
  'HostId': 'GXRh2SlBNfMR5ffmBOCNi2ZsURR4ahoVSjlilqVOl8wr1ehOM1VAr5L/xmoD7yIe/BuGGgkSabc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'GXRh2SlBNfMR5ffmBOCNi2ZsURR4ahoVSjlilqVOl8wr1ehOM1VAr5L/xmoD7yIe/BuGGgkSabc=',
   'x-amz-request-id': '3F2BB1912F5427B8',
   'date': 'Wed, 07 Nov 2018 00:03:22 GMT',
   'etag': '"6a03b1c6a6566aacc398c58b9b84173d"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"6a03b1c6a6566aacc398c58b9b84173d"'}